### Statistical Analysis from <i>Penetrating Abdominal Trauma with Hollow Viscus Injury: Infectious Risks Revisited<i/>

David T. Pointer Jr, MD, Lili E. Schindelar, BS, Matthew D. Zelhart, MD, Zaid Al-Qurayshi, MD, MPH, Douglas P. Slakey, MD, MPH, FACS, Ronald L. Nichols, MD, MS, FACS

Department of Surgery, Tulane University School of Medicine, New Orleans, LA, USA

**Background:** <i> Surgical site infections (SSI) have a great impact on morbidity and mortality in the surgical patient. The Eastern Association for the Surgery of Trauma has established guidelines for the use of prophylactic antibiotics in penetrating abdominal trauma. We aimed to determine SSI associated risk factors in patients sustaining penetrating abdominal trauma with hollow viscus injury, analyze perioperative antibiotic type and administration, and determine adherence to recommended guidelines.<i/>

In [108]:
import pandas as pd
import numpy as np

#import cleaned data and check the data/column names
df = pd.read_csv("trauma_cleaned.csv")
#df.describe()
#make this more intuitive
df['survival'] = df.died == 1
df.keys()

Index(['id', 'mrn', 'act_no', 'admit_date', 'age', 'gender', 'wt', 'race',
       'inj_type', 'inj_site', 'sbp', 'dbp', 'rr', 'pr', 'o2', 'gcs', 'shock',
       'dispos', 'proc_date', 'time', 'inj_org', 'Bld_unit', 'ostomy', 'wound',
       'antibio', 'dose1', 'day1', 'day2', 'day_beyond', 'ssi', 'ssi_org',
       'absc', 'absc_org', 'pneu', 'septic', 'sept_org', 'cdiff', 'inf2',
       'died', 'comp', 'los', 'infprop', 'survprop', 'survival'],
      dtype='object')

### VAC Patients
Was there a significant difference between group 3 and groups 1 and 2 for the following:
* SSI (ssi = 'Y')
* Abscess (absc = 'Y')
* Sepsis (septic = 'Y')
* c diff infection (cdiff = 'Y')
* Survival (survival = True)
* Days in Hospital (sol is a continuous variable, use student's T-test)


In [109]:
from scipy.stats import fisher_exact
from scipy.stats import ttest_ind

#VAC patients from wound = 3
#VAC vs SSI
table = pd.crosstab(df.ssi =='Y',df.wound == 3)
print("VAC (wound = True) vs SSI (True)")
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#VAC vs Abscess
print("VAC (wound = True) vs Abscess (True)")
table = pd.crosstab(df.absc =='Y',df.wound == 3)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#VAC vs Sepsis
print("VAC (wound = True) vs Sepsis (True)")
table = pd.crosstab(df.septic =='Y',df.wound == 3)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#VAC vs c diff Infection
print("VAC (wound = True) vs c diff Infection (True)")
table = pd.crosstab(df.cdiff =='Y',df.wound == 3)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#VAC vs Survival
print("VAC (wound = True) vs Survival (True)")
table = pd.crosstab(df.survival ==True,df.wound == 3)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")


#VAC vs Days in Hospital
vac = df[df['wound']==3]
non_vac = df[df['wound']!=3]
print("Average Days in the Hospital per Wound Type (VAC = 3)")
print(pd.pivot_table(df, values='los', columns=['wound']))
print("p_value: ",ttest_ind(vac['los'], non_vac['los'])[1])


VAC (wound = True) vs SSI (True)
wound  False  True 
ssi                
False    102     41
True      15     20
p-value:  0.00253025884135
____________________
VAC (wound = True) vs Abscess (True)
wound  False  True 
absc               
False     99     38
True      18     23
p-value:  0.00128251397465
____________________
VAC (wound = True) vs Sepsis (True)
wound   False  True 
septic              
False     110     49
True        7     12
p-value:  0.00892667579243
____________________
VAC (wound = True) vs c diff Infection (True)
wound  False  True 
cdiff              
False    116     57
True       1      4
p-value:  0.047621479644
____________________
VAC (wound = True) vs Survival (True)
wound     False  True 
survival              
False         5      8
True        112     53
p-value:  0.0638045652007
____________________
Average Days in the Hospital per Wound Type (VAC = 3)
wound
1.0    11.043103
2.0    12.000000
3.0    36.737705
Name: los, dtype: float64
p_value:  3.80444095

Wound type is statistically significant in all of the above cases. However, at 95% confidence, we cannot reject the null hypothesis for survival.

### SSI superficial vs Abscess   
Was there a difference between superficial Surgical Site Infections vs Abscess in the following:
*	Survival  (survival = True)
*	Sepsis (septic = 'Y') <br> <br>
Note: Superficial SSI is denoted by ssi = 'Y' AND absc = 'N'; Abscess denoted by absc = 'Y'

In [111]:
def sssi_vs_abscess (row):
    if row['ssi'] == 'Y' and row['absc'] == 'N':
        return 's'
    if row['absc'] == 'Y':
        return 'a'
    return 'other'
df['superficial_ssi'] = df.apply (lambda row: sssi_vs_abscess(row),axis=1)

#make sure that worked
print(df[['ssi','absc','superficial_ssi']].head()) #nice :)

sub_df = df[df.superficial_ssi != 'other']
print("\n")
print("Now we are only looking at " + str(len(sub_df)) + " patients")
print("\n")

#Superficial SSI vs Survival
print("Superficial SSI (s_vs_a = True) vs Survival (True)")
table = pd.crosstab(sub_df.survival ==True,sub_df.superficial_ssi == 's')
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Superficial SSI vs Sepsis
print("Superficial SSI (s_vs_a = True) vs Sepsis (True)")
table = pd.crosstab(sub_df.septic =='Y',sub_df.superficial_ssi == 's')
print(table)
print("p-value: ", fisher_exact(table)[1])
print("______________________")


  ssi absc superficial_ssi
0   Y    Y               a
1   Y    N               s
2   N    N           other
3   N    N           other
4   N    N           other


Now we are only looking at 62 patients


Superficial SSI (s_vs_a = True) vs Survival (True)
superficial_ssi  False  True 
survival                     
False                0      2
True                41     19
p-value:  0.111052353252
____________________
Superficial SSI (s_vs_a = True) vs Sepsis (True)
superficial_ssi  False  True 
septic                       
False               33     15
True                 8      6
p-value:  0.524092004229
______________________


Neither of these tests are statistically significant

### Use of Antibiotics > 1 day vs. 1 day or less 
(Note: if a patient has **any** prescriptions for $2^+$ days then they are incuded in Antibiotics > 1 days)

* SSI (ssi = 'Y')
* Abscess (absc = 'Y')
* Sepsis (septic = 'Y')
* c diff infection (cdiff = 'Y')
* Survival (survival = True)
* Days in Hospital (sol is a continuous variable, use student's T-test)

In [113]:
boolean_df = df.isnull()

def days (row):
    if row['day2'] == False or row['day_beyond'] == False:
        return 2
    if row['dose1'] == False or row['day1'] == False:
        return 1
    return 0
df['days'] = boolean_df.apply (lambda row: days(row),axis=1)

#make sure that worked
print(df[['dose1','day1','day2','day_beyond','days']].head()) #nice :)
#quickly check how many patients in each group
print("\n")
print("Antibiotics >1 days: ",len(df[df.days==2]))
print("Antibiotics 1 day: ",len(df[df.days!=2]))

#4 patients did not receive antibiotics
print("\n")
print("Taking these 4 patients who received no antibiotics out of the analysis:")
print(df[['dose1','day1','day2','day_beyond','days']][df.days==0])
#so take them out of analysis
anti_df = df[df.days!=0]

  dose1     day1 day2 day_beyond  days
0   NaN  Mefoxin  NaN        NaN     1
1   NaN  Mefoxin  NaN      Ancef     2
2   NaN    Ancef  NaN        NaN     1
3   NaN  Mefoxin  NaN        NaN     1
4   NaN    Ancef  NaN    Mefoxin     2


Antibiotics >1 days:  55
Antibiotics 1 day:  123


Taking these 4 patients who received no antibiotics out of the analysis:
    dose1 day1 day2 day_beyond  days
80    NaN  NaN  NaN        NaN     0
106   NaN  NaN  NaN        NaN     0
131   NaN  NaN  NaN        NaN     0
167   NaN  NaN  NaN        NaN     0


In [114]:
#Days vs SSI
table = pd.crosstab(anti_df.ssi =='Y',anti_df.days == 2)
print("Days (days = True means 2+ days) vs SSI (True)")
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Days vs Abscess
print("Days (days = True means 2+ days) vs Abscess (True)")
table = pd.crosstab(anti_df.absc =='Y',anti_df.days == 2)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Days vs Sepsis
print("Days (days = True means 2+ days) vs Sepsis (True)")
table = pd.crosstab(anti_df.septic =='Y',anti_df.days == 2)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Days vs c diff Infection
print("Days (days = True means 2+ days) vs c diff Infection (True)")
table = pd.crosstab(anti_df.cdiff =='Y',anti_df.days == 2)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Days vs Survival
print("Days (days = True means 2+ days) vs Survival (True)")
table = pd.crosstab(anti_df.survival ==True,anti_df.days == 2)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")


#Days vs Days in Hospital
multiple = anti_df[anti_df['days']==2]
one = anti_df[anti_df['days']==1]
print("Average Days in the Hospital per Group (days = 2+ vs days = 1)")
print(pd.pivot_table(anti_df, values='los', columns=['days']))
print("p_value: ",ttest_ind(vac['los'], non_vac['los'])[1])

Days (days = True means 2+ days) vs SSI (True)
days   False  True 
ssi                
False    101     38
True      18     17
p-value:  0.0242178939665
____________________
Days (days = True means 2+ days) vs Abscess (True)
days   False  True 
absc               
False     89     44
True      30     11
p-value:  0.565128440384
____________________
Days (days = True means 2+ days) vs Sepsis (True)
days    False  True 
septic              
False     111     44
True        8     11
p-value:  0.0163258999652
____________________
Days (days = True means 2+ days) vs c diff Infection (True)
days   False  True 
cdiff              
False    116     53
True       3      2
p-value:  0.651793652703
____________________
Days (days = True means 2+ days) vs Survival (True)
days      False  True 
survival              
False         9      3
True        110     52
p-value:  0.754807000371
____________________
Average Days in the Hospital per Group (days = 2+ vs days = 1)
days
1    16.302521
2    28.5

The two tests here that are significant are Sepsis and SSI (patients with only one dose/day of antibiotics are associated with fewer cases of Sepsis and SSI).
